# Simple Group Lasso Program

In this digression, the idea is to illustrate how one might program a functional (and perhaps crude) group Lasso program in `Stata`\`Mata`. Most of what follows is done using the material described in [Farrell (2015)](https://arxiv.org/abs/1309.4686). In addition to containing the theoretical justification for using a group lasso in conjunction with a doubly-robust estimator of treatment effects, this paper also contains many practical details.

### Coefficients and the Lasso

The grouped lasso is useful in circumstances where one is effectively fitting a series of regressions/equations, but wishes to maintain consistency across variables used in the regressions. So, a leading application of a grouped lasso is one in which the lasso is set up to let coefficients across all regressions be zero or positive. 

Suppose that coefficients fall into different groups, and group $g$ has a total of $N_g$ coefficients, with coefficient group $g$ as $\beta_g=\beta_{1g},\beta_{2g},...,\beta_{N_gg}$. While the groups may be grouped regression coefficients, they might also be dummies that go together in an obvious manner; education or income levels, age groups, etc.

The usual lasso tacks on a penalty function of the form $(|\beta_1| + |\beta_2| + ... )$ to the objective function. The group lasso uses a mixed $\ell_1\ell_2$ norm, as described in [Yuan and Lin (2006)]( http://onlinelibrary.wiley.com/doi/10.1111/j.1467-9868.2005.00532.x/abstract), so if we have $G$ groups with $N_g, g=1,2,...,G$ coefficients in them, we have the penalty function:

$$
P(\beta)=w_1\sqrt{\left(|\beta_{11}|+|\beta_{21}|+...+|\beta_{N_11}|\right)^2} +w_2\sqrt{\left(|\beta_{12}|+|\beta_{22}|+...+|\beta_{N_22}|\right)^2}+...+w_G\sqrt{\left(|\beta_{1G}|+|\beta_{2G}|+...+|\beta_{N_GG}|\right)^2}
$$

The weights $w_1,w_2,...,w_{NG}$ are usually taken to be $\sqrt{N_g}$ to also take into account group size.

For any trivial group - a group with a sole parameter - the group lasso collapses to the the usual lasso. For groups with multiple parameters, however, the above penalty scheme forces all the parameters in the group to be different than zero or zero.


## Objective function 

The modified objective function to be minimized is something like:

$$
L = f(y,x,\beta)- \lambda P(\beta)
$$

Where $f(y,x,\beta)$ could be just about anything (the negative of a likelihood perhaps, or some other thing of even a simple quadratic like $-(y-X\beta)'(y-X\beta)$). This is maximized given the lasso penalty. So, one must decide on $\lambda$, which depends upon circumstances. We will get in to this a bit below, following  [Farrell (2015)](https://arxiv.org/abs/1309.4686). 

## Some Data

We begin the mock up of the group lasso using `ipystata`. The ideas here will be extended and applied on the fly as I fit actual models. 

The last line of code renders a global macro for ease of passing around extended variable lists. 

In [1]:
import ipystata

Terminated 1 unattached Stata session(s).


I will start with the standard **`auto.dta`** data that comes with Stata, and form a model of whether or not a car is foreign using things like its price, repair record, size, gas mileage, etc. I'll add in a few categorical variables as this is a situation in which the group lasso is often applied. 

Following [Farrell (2015)](https://arxiv.org/abs/1309.4686) **Remark 4, page 10**, it helps to standardize the variables. One does not wish that units of measure impact a decision to include a coefficient. As I standardize, I also get the maximum (absolute) value of an observation, and the number of observations, because these both figure into the calcululation of the penalty weight $\lambda$ [Farrell (2015)](https://arxiv.org/abs/1309.4686) describes. 

In [2]:
%%stata -s gl
sysuse auto, clear

drop if rep == 0

tab rep78, gen(rdum)                       // Generate some categorical variables

egen dispcat = cut(displacement), at(0, 100, 250, 500)   // Another categorical variable
tab dispcat, gen(dispdum)

egen weightcat = cut(weight), at(0, 2250, 3200, 4000, 10000)  // Yet another categorical variable
tab weightcat, gen(wedum)

global contvars price rep78 mpg headroom weight length turn gear_ratio

scalar Max = 0
foreach v of global contvars {
    quietly sum `v'
    replace `v' = (`v' - r(mean))/r(sd)
    sum `v'
    scalar test = max(abs(r(min)),abs(r(max)))
    if test > Max {
        scalar Max = test
    }
}

scalar N = r(N)

global yvar foreign
global xlist $contvars wedum1 wedum2 wedum3 rdum1 rdum2 rdum3 dispdum1 dispdum2
disp "$xlist"


(1978 Automobile Data)

(0 observations deleted)

     Repair |
Record 1978 |      Freq.     Percent        Cum.
------------+-----------------------------------
          1 |          2        2.90        2.90
          2 |          8       11.59       14.49
          3 |         30       43.48       57.97
          4 |         18       26.09       84.06
          5 |         11       15.94      100.00
------------+-----------------------------------
      Total |         69      100.00

    dispcat |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |         14       18.92       18.92
        100 |         41       55.41       74.32
        250 |         19       25.68      100.00
------------+-----------------------------------
      Total |         74      100.00

  weightcat |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |         19       25.68       25.68
       2250 |         18  

We now have a series of variables which we can use in our mock-up analysis, some of which fall into groups: (`weightcat`, `dispcat`, e.g.). 

### Choice of Lambda 

Since I am working with discrete outcomes, I set the $\lambda$ parameter according to what  [Farrell (2015)](https://arxiv.org/abs/1309.4686) suggests when the model is discrete/categorical. In this case, he suggests setting $\lambda$ as:

$$
\lambda_D = \frac{2\mathcal{X}\sqrt{\mathcal{T}}}{\sqrt{n}}\left(1+\frac{\log(p \vee n)^{3/2+\delta_D}}{\sqrt{\mathcal{T}}}\right)^{1/2}
$$

In the above, $\mathcal{X}$ is the maximum absolute value of any of the variables in the sample. Since I have standardized all the variables, this shouldn't be much bigger than $2$; I set it at 3.4 to be safe (this is the maximum value in the above standardized variable set). $\mathcal{T}$ is the number of treatments, which is just unity (more would mean we have something like a multinomial logistic model and multiple treatments). Since $n$ is larger than $p$, and $n=74$, we have $p \vee n = 74$. And to be safe, I set $\delta_D=1$. 

In the implementation, I'm going to use a "grouping" matrix to collect coefficients belonging to the same group. We will also print out the calculated components of $\lambda_D$, just to see that they are sensible. Here goes:

# A Mata-based treatment of the problem

`Mata` admits more control over the whole process, and also is a bit more compact in the coding. So, here is a Mata implementation. Really, all that one needs to do is look at the resulting coefficient estimates and see which ones are effectively zero and which ones are not. Once this step is done, I can then apply the usual Stata tools in estimating the model. 

### Additional comments:

1. The objective function is a little spiky, so it is helpful to use the `hybrid` option for the Hessian. If there are a large number of coefficients, it helps to use Nelder-Mead optimization followed by the usual methods. 
2. Setting `maxiter` is also helpful. The optimization routine sometimes finds the minimum, but doesn't recognize it because $H$ is singular at this point. 


In [3]:
%%stata -s gl

mata:
    st_view(X=.,.,"$xlist")
    st_view(y=.,.,"$yvar")
    
    lambda = 2*max(abs(X))/sqrt(rows(X))*(1+ln(rows(X))^(3/2+1))^(1/2)

    g = 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0 \
        0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0 \
        0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0 \
        0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0 \
        0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0 \
        0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0 \ 
        0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0 \
        0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0 \
        0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0 \
        0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0 \
        0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
end

mata:
    void gll_mata(M, todo, b, f, gr, H) {
        
        y  = moptimize_util_depvar(M, 1)
        xb = moptimize_util_xb(M, b, 1)
        
        lam = moptimize_util_userinfo(M, 1)
        g   = moptimize_util_userinfo(M, 2)
 
        bt  = b[1::cols(b) - 1]
        gb  = sqrt(rowsum(g)):*sqrt(rowsum((bt:^2):*g))
        norm = colsum(gb)       
       
        lnf  = (y :== 1) :*xb - ln(1 :+ exp(xb))       
        
        f = sum(lnf) - norm*lam
    }
end    



------------------------------------------------- mata (type end to exit) ----------------------
note: argument todo unused
note: argument gr unused
note: argument H unused



In [4]:
%%stata -s gl -os

mata:
    M = moptimize_init()
    moptimize_init_evaluator(M, &gll_mata())
    moptimize_init_evaluatortype(M, "d0")
    moptimize_init_eq_indepvars(M, 1, X)
    moptimize_init_depvar(M, 1, y)
    moptimize_init_userinfo(M, 1, lambda)
    moptimize_init_userinfo(M, 2, g)
    moptimize_init_singularHmethod(M, "hybrid")
    moptimize_init_conv_maxiter(M, 500)
    moptimize_init_trace_value(M, "off")
    moptimize(M)
    moptimize_result_display(M)
end
 

------------------------------------------------- mata (type end to exit) ----------------------
                                                Number of obs     =         74

------------------------------------------------------------------------------
           y |      Coef.   Std. Err.      z    P>|z|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
       price |   2.06e-07   .0002986     0.00   0.999    -.0005851    .0005855
       rep78 |    .599299   .4103106     1.46   0.144    -.2048951    1.403493
         mpg |   1.28e-08   .0001841     0.00   1.000    -.0003607    .0003608
    headroom |  -8.96e-08   .0001904    -0.00   1.000    -.0003733    .0003731
      weight |  -1.16e-07   .0004639    -0.00   1.000    -.0009093     .000909
      length |  -1.03e-06   .0006823    -0.00   0.999    -.0013383    .0013362
        turn |  -.4514483   .5245564    -0.86   0.389     -1.47956    .5766634
  gear_ratio |   1.103175   .4893

One can see from the above output that `rep78`, `turn`, and `gear_ratio` are picked out by the Lasso as being variables with enough explanatory power for inclusion in the model, with the rest of the coefficients/grouped terms effectively having coefficients of zero.

# Program #2: Lasso with multiple treatments

It also helps to have a few methods available for a multi-leveled situation in which there are multiple treatments/outcomes. The idea here is to group coefficients across multiple linear forms that may appear in a model. Here is a group lasso program with multiple treatments, trinomial logit outcomes, and other stuff. 

Note that the key challenge is unpacking `Mata`'s coefficient vector $b$ into its two components, which have equal numbers of parts. I simply split it down the middle and then consider the coefficient vector in its entirety without the very last entry, as this is the constant. 

In [5]:
%%stata -s gl
mata:
    void gltril_mata(M, todo, b, f, gr, H) {
        
        y   = moptimize_util_depvar(M, 1)
        xb1 = moptimize_util_xb(M, b, 1)
        xb2 = moptimize_util_xb(M, b, 2)

        lam = moptimize_util_userinfo(M, 1)
        g   = moptimize_util_userinfo(M, 2)
 
        b1  = b[1::cols(b)/2]
        b2  = b[cols(b)/2 + 1::cols(b)]

        b1 = b1[1::cols(b1)-1]
        b2 = b2[1::cols(b2)-1]

        bt = b1, b2

        gb  = sqrt(rowsum(g)):*sqrt(rowsum((bt:^2):*g))
        norm = colsum(gb)       
    
        lnf  = (y :== 1) :*xb1 + (y :==2 ) :*xb2 - ln(1 :+ exp(xb1) :+ exp(xb2))       
        
        f = sum(lnf) - norm*lam
    }
end   

------------------------------------------------- mata (type end to exit) ----------------------
note: argument todo unused
note: argument gr unused
note: argument H unused



# A test run of the new program

Here, we test out the new program using a different data set. This is the `mlogit` data set that one can get online from the below-referenced website. I also followed some of the documentation available at [this site](https://stats.idre.ucla.edu/stata/output/multinomial-logistic-regression/).

In [6]:
%%stata -s gl

use https://stats.idre.ucla.edu/stat/stata/output/mlogit, clear

describe    
    
recode ice_cream (1 = 0) (2 = 1) (3 = 2), generate(ice)    
tab ice

mlogit ice video puzzle female

(highschool and beyond (200 cases))

Contains data from https://stats.idre.ucla.edu/stat/stata/output/mlogit.dta
  obs:           200                          highschool and beyond (200 cases)
 vars:             5                          3 Apr 2007 14:04
 size:         4,000                          
------------------------------------------------------------------------------------------------
              storage   display    value
variable name   type    format     label      variable label
------------------------------------------------------------------------------------------------
id              float   %9.0g                 
female          float   %9.0g      fl         gender
ice_cream       float   %10.0g     ic         favorite flavor of ice cream
video           float   %9.0g                 score on video game
puzzle          float   %9.0g                 score on puzzle game
---------------------------------------------------------------------------------------------

To try the multiple lasso program, compute a series of interactions that we wish to pare down in fitting the actual model:

In [7]:
%%stata -s gl

gen video2    = video*video
gen puzzle2   = puzzle*puzzle

gen vidpuz    = video*puzzle
gen vidfem    = video*female
gen vidpuzfem = video*puzzle*female

global xlist video puzzle female video2 puzzle2 vidpuz vidfem vidpuzfem

# Standardizing the variables

As noted above, it is useful to standardize all the variables so the actual units don't wind up entering into the analysis.

In [8]:
%%stata -s gl

scalar Max = 0
foreach v of global xlist {
    quietly sum `v'
    replace `v' = (`v' - r(mean))/r(sd)
    sum `v'
    scalar test = max(abs(r(min)),abs(r(max)))
    if test > Max {
        scalar Max = test
    }
}

scalar N = r(N)

disp N 
disp Max


. foreach v of global xlist {
  2.     quietly sum `v'
  3.     replace `v' = (`v' - r(mean))/r(sd)
  4.     sum `v'
  5.     scalar test = max(abs(r(min)),abs(r(max)))
  6.     if test > Max {
  7.         scalar Max = test
  8.     }
  9. }
(200 real changes made)

    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
       video |        200   -2.94e-09           1  -2.610876   2.237172
(200 real changes made)

    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
      puzzle |        200   -7.38e-09           1  -2.459529   1.732056
(200 real changes made)

    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
      female |        200   -2.24e-08           1  -1.091702   .9114209
(200 real changes made)

    Vari

## Recalculating grouping matrix and $\lambda$

We now have to recompute $\lambda$, as our data is a bit different. The key is that now our number of treated groups is $\mathcal{T}=2$, whereas it was just unity in the simple multinomial logit model. For ease of exposition, let's reproduce the formula here:

$$
\lambda_D = \frac{2\mathcal{X}\sqrt{\mathcal{T}}}{\sqrt{n}}\left(1+\frac{\log(p \vee n)^{3/2+\delta_D}}{\sqrt{\mathcal{T}}}\right)^{1/2}
$$

In this case, we haven't really grouped variables, so we just need an identity matrix as our "grouping" matrix, which conforms with the number of variables. Note how the grouping matrix has to be expanded so it has the form:

$$
\begin{array}{cccccc}
1 & 0 & 0 & 1 & 0 & 0 \\
0 & 1 & 0 & 0 & 1 & 0 \\
0 & \dots
\end{array}
$$

Thus it picks out the first coefficient in the first equation and the first in the second, etc.


In [10]:
%%stata -s gl -os
global xlist video puzzle female video2 puzzle2 vidpuz vidfem vidpuzfem
global yvar ice
mata:
    st_view(X=.,.,"$xlist")
    st_view(y=.,.,"$yvar")
    
    lambda = 2*max(abs(X))*sqrt(2)/sqrt(rows(X))*(1+ln(rows(X))^(3/2+1)/sqrt(2))^(1/2)   
    
    g = J(1,2,1)#I(cols(X))
end

mata:
    M = moptimize_init()
    moptimize_init_evaluator(M, &gltril_mata())
    moptimize_init_evaluatortype(M, "d0")
    moptimize_init_eq_indepvars(M, 1, X)
    moptimize_init_eq_indepvars(M, 2, X)
    moptimize_init_depvar(M, 1, y)
    moptimize_init_userinfo(M, 1, lambda)
    moptimize_init_userinfo(M, 2, g)
    moptimize_init_conv_maxiter(M, 500)
    moptimize_init_singularHmethod(M, "hybrid")
    moptimize_init_trace_value(M, "off")
    moptimize(M)
    moptimize_result_display(M)
end   


convergence not achieved

                                                Number of obs     =        200

------------------------------------------------------------------------------
           y |      Coef.   Std. Err.      z    P>|z|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
eq1          |
       video |   5.49e-06   .0016521     0.00   0.997    -.0032326    .0032435
      puzzle |   2.40e-06   .0011711     0.00   0.998    -.0022928    .0022976
      female |  -.1617904   .1825151    -0.89   0.375    -.5195135    .1959327
      video2 |   8.96e-07   .0008129     0.00   0.999    -.0015924    .0015942
     puzzle2 |   .0462398   .2139496     0.22   0.829    -.3730938    .4655733
      vidpuz |   .2994924   .2591366     1.16   0.248    -.2084059    .8073908
      vidfem |  -.0000125   .0023082    -0.01   0.996    -.0045365    .0045116
   vidpuzfem |  -3.98e-06   .0013142    -0.00   0.998    -.0025797    .0025717
       _c

# Results

From the results, one can see how the group lasso has worked on the estimated model. The results suggest that for the grouped logit, `video`, `puzzle`, `video2`, `vidfem`, and `vidpuzfem` can be dropped to get a more parsimonious model.

### Going Forward

In the interests of transparency, I shall include `Mata` code for the lasso program in the text. 